In [7]:
from myMongoClient import *
import pandas as pd
%reload_ext autoreload
%autoreload 2

db = "documents-100039"
organisation_id = int(db[-6:])
mongo_client = MyMongoClient(db)

## Get a mapping form the file_name and the type of document it is

In [6]:
label_versions = pd.read_csv("data/labels_results/2024-01-18_16:15:33_df_gold_labels.csv")
output = label_versions[label_versions['label_name'] == 'document type'][['doc_id', 'label_value']]
doc_hash_value = {rows['doc_id'] : rows['label_value'] for idx , rows in output.iterrows()}
file_name_doc_hash = {inst['files'][0]['fileName'] : inst['_id'] for inst in mongo_client.get_all_documents() if inst['files'] and inst['files'][0]['fileName'].startswith(('Pseudo_',"pseudo_"))}
file_nam_value = {key : doc_hash_value[value] for key, value in file_name_doc_hash.items() if value in doc_hash_value}
# file_nam_value, len(file_nam_value)

dump(file_nam_value, "doc_type_from_file_name.pkl")

## Get gold labels from prompt

In [ ]:
load("data/saved_prompts/prompts_2024-01-23_13:38:13.csv")